In [1]:
import warnings #library to ignore warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets #widgets for voila deployment
from IPython.display import display, clear_output

In [2]:
import requests # for extracting html content
import time
import os
import pandas as pd
from csv import reader
import numpy as np

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import json

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
import sklearn
import statsmodels.api as sm 
import scipy.stats as st
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
sns.set()

In [5]:
#!pip install voila
#!pip install widgetsnbextension
#!pip uninstall ipywidgets --only in terminal
#!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling: voila
- Writing config: D:\anaconda3\etc\jupyter
    - Validating...
      voila 0.4.0 ok


In [6]:
#Creating widgets for the deployement

#Dropdown widget
location = widgets.Dropdown(
    options=['India','United States'],
    value='India',
    description='Location:',
)

job_profile = widgets.Dropdown(
    options=['Data scientist','Software engineer'],
    value='Data scientist',
    description='Job Profile:',
)

#Text Widget
profiles_required = widgets.Text(placeholder='Enter no of profiles',description='# of Profiles:')


LOB = widgets.Dropdown(
    options=['LOB_BFSI','LOB_CSMP','LOB_EAS','LOB_ERS','LOB_ETS','LOB_Healthcare','LOB_INFRA','LOB_MMS'],
    value='LOB_BFSI',
    description='LOB:',
)


Offered_band = widgets.Dropdown(
    options=['Offered_band_E1','Offered_band_E2','Offered_band_E3'],
    value='Offered_band_E1',
    description='CTC Band:',
)

Candidate_ID = widgets.Text(placeholder='Candidate_ID',description='Candidate_ID')

Profile_Link = widgets.Text(placeholder='Profile Link',description='Profile Link')

Duration_to_accept_offer = widgets.Text(placeholder='Duration_to_accept_offer',description='Duration_to_accept_offer')

Notice_period = widgets.Text(placeholder='Notice_period',description='Notice_period')

Percent_difference_CTC = widgets.Text(placeholder='Percent_difference_CTC',description='Percent_difference_CTC')

Rex_in_Yrs = widgets.Text(placeholder='Rex_in_Yrs',description='Rex_in_Yrs')

Companies_Count = widgets.Text(placeholder='Companies_Count',description='Companies_Count')

Proximity_to_Job_Location = widgets.Text(placeholder='Proximity_to_Job_Location',description='Proximity_to_Job_Location')

Age = widgets.Text(placeholder='Age',description='Age')

Match_Score = widgets.Text(placeholder='Match_Score',description='Match_Score')

DOJ_Extended_Yes = widgets.Text(placeholder='DOJ_Extended_Yes',description='DOJ_Extended_Yes')

Joining_Bonus_Yes = widgets.Text(placeholder='Joining_Bonus_Yes',description='Joining_Bonus_Yes')

Candidate_relocate_actual_Yes = widgets.Text(placeholder='Candidate_relocate_actual_Yes',description='Candidate_relocate_actual_Yes')

Gender_Male = widgets.Text(placeholder='Gender_Male',description='Gender_Male')

In [7]:
#Defining function to fetch linkedin urls

def get_urls(country_name,profile_name):
    Country_name = country_name
    Job_Profile = profile_name
    
    search_query = "site:linkedin.com/in AND" + Job_Profile + " AND " + Country_name
    # Use a try-except block to handle the ImportError
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
    
    profile_link = []
    for i, link in enumerate(search(search_query, num=5, stop=5, pause=2)):
        # Check if the link is not already in the list
        if link not in profile_link:
            profile_link.append(link)
            #print(f"{i+1}. {link}")


    # Use pandas to write the list to a CSV file
    column_names = ['LinkedIn_Profile_Links']
    df = pd.DataFrame(profile_link,columns=column_names)
    df['Candidate ID'] = df.index + 100
    df['LinkedIn_Profile_Links'] = df['LinkedIn_Profile_Links'].replace({'in.l': 'www.l'}, regex=True)
    df.to_csv(r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\Outputs\linkedin_profile_links.csv',index = False)

    

In [8]:

#defining the function to fetch data from linkedin

def get_scrapped_data():
    
    PAGE_LOAD_TIME_WAIT_SEC = 20
    CALL_WAIT_TIME_SEC = 10

    EXP_PAGE_PATH = "/details/experience"
    SKILLS_PAGE_PATH = "/details/skills"

    INPUT_PROFILES_FILE_PATH = r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\Outputs\linkedin_profile_links.csv'
    PROFILE_DF_FILE = r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\Outputs\linkedin_profile_dataframe.csv'
    PROFILE_JSON_FILE = r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\Outputs\linkedin_profile_json.csv'

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    # driver.get method() will navigate to a page given by the URL address
    driver.get('https://www.linkedin.com/')

    # locate email form by_class_name
    #username = driver.find_element(By.CLASS_NAME,'input__input')
    username = driver.find_element("name",'session_key')

    # send_keys() to simulate key strokes
    username.send_keys('gunika.kamra.ampba@gmail.com')

    password = driver.find_element("name",'session_password')
    password.send_keys('isb@12345')

    log_in_button = driver.find_element(By.CLASS_NAME,'sign-in-form__submit-button')
    log_in_button.click()
    
    
    
    profile_df = pd.read_csv(INPUT_PROFILES_FILE_PATH)

    #Fetching name from the linkedin profile
    def get_name_from_profile(soup):

        if (soup.find('section', {'class':"artdeco-card ember-view pv-top-card"}) is None):
            return ""

        try:
            name = soup.find('section', {'class':"artdeco-card ember-view pv-top-card"}) \
               .find('div',{'class':'mt2 relative'})\
               .find('h1',{'class':'text-heading-xlarge inline t-24 v-align-middle break-words'}).string.strip()
            #print(name)
            return name
        except:
            return ""
        
        
#fetching location from the linkedin profile
    def get_location_from_profile(soup):
        location = ""

        if ((soup.find('section', {'class':"artdeco-card ember-view pv-top-card"})) is None):
            return location
        try:
            location = soup.find('section', {'class':"artdeco-card ember-view pv-top-card"}) \
               .find('div',{'class':'mt2 relative'})\
               .find('div',{'class':'pv-text-details__left-panel mt2'})\
               .find('span',{'class':'text-body-small inline t-black--light break-words'}).string.strip()
            #print(location)
        except:
            pass

        return location
    

#fetching descritions and about sections
    def get_about_from_profile(soup,text_bagofwords):
        about_text = ""

        try:
            cards =soup.find_all('section', {'class':"artdeco-card ember-view relative break-words pb3 mt2"})
        except:
            return about_text,text_bagofwords

        for card in cards:
            if (card.find('div',attrs={'id':'about'})):
                try:
                    about_div = card.find('div',attrs={'class':'inline-show-more-text inline-show-more-text--is-collapsed full-width'})
                    about_text = re.sub(r'<.*>',"",str(about_div))
                except:
                    pass

        text_bagofwords += about_text
        return about_text, text_bagofwords

    
#fetching education background
    def get_about_education_from_profile(soup,text_bagofwords):

        educ_list = []

        try:
            cards =soup.find_all('section', {'class':"artdeco-card ember-view relative break-words pb3 mt2"})
        except:
            return educ_list

        for card in cards:
            if (card.find('div',attrs={'id':'about'})):
                try:
                    about_div = card.find('div',attrs={'class':'inline-show-more-text inline-show-more-text--is-collapsed full-width'})
                    about_text = re.sub(r'<.*>',"",str(about_div))
                    text_bagofwords += about_text
                except:
                    pass

            if(card.find('div',attrs={'id':'education'})):
                edu_list = card.find_all('li',{'class':'artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})

                for edu in edu_list:
                    educ_dict = {}
                    collegename_string = ""
                    degreename_string = ""
                    degreedate_string = ""

                    try:
                        collegename_div = edu.find('div',{'class':'display-flex flex-column full-width align-self-center'})\
                                        .find('span',{'class':'mr1 hoverable-link-text t-bold'})\
                                        .find('span',{'class':'visually-hidden'})
                        collegename_string = re.sub('<.*?>',"",str(collegename_div))
                    except:
                        pass

                    try:
                        degreename_div = edu.find('div',{'class':'display-flex flex-column full-width align-self-center'})\
                                        .find('span',{'class':'t-14 t-normal'})\
                                        .find('span',{'class':'visually-hidden'})
                        degreename_string = re.sub('<.*?>',"",str(degreename_div))
                    except:
                        pass

                    try:
                        degreedate_div = edu.find('div',{'class':'display-flex flex-column full-width align-self-center'})\
                                        .find('span',{'class':'t-14 t-normal t-black--light'})\
                                        .find('span',{'class':'visually-hidden'})
                        degreedate_string = re.sub('<.*?>',"",str(degreedate_div))
                    except:
                        pass

                    educ_dict["Name"]=collegename_string
                    educ_dict["Degree"]=degreename_string
                    educ_dict["Date"]=degreedate_string
                    educ_list.append(educ_dict)

        return about, educ_list, text_bagofwords
    


    #Get Experience List
    def get_experience_from_experience_page(soup,text_bagofwords):
        job_list = []
        total_exp_months = 0
        companies_count = 0

        if(soup.find('section',{'class':'artdeco-card ember-view pb3'})is None):
            return total_exp_months, companies_count, text_bagofwords, job_list

        try:
            exp_list = soup.find('section',{'class':'artdeco-card ember-view pb3'})\
                    .find('div',{'class':'pvs-list__container'})\
                    .find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})
        except:
            return total_exp_months, companies_count, text_bagofwords, job_list

        companies_count = len(exp_list)
        for job in exp_list:

            #Different layouts depending on whether multiple roles within same company or 1 role in the company
            if (job.find('div',{'class':'display-flex flex-column full-width'})):
                job_dict ={}
                designation = ""
                company = ""
                tenure = ""
                location = ""
                description_string = start_date = end_date = ""

                #Designation field crawled
                try:
                    designation_div = job.find('div',{'class':'display-flex flex-column full-width'})\
                            .find('span',{'class':'mr1 t-bold'})\
                            .find('span',{'class':'visually-hidden'})
                    designation = re.sub('<.*?>',"",str(designation_div))
                except:
                    pass
                job_dict["Designation"] = designation

                #Company field crawled
                try:
                    company_div = job.find('div',{'class':'display-flex flex-column full-width'})\
                                .find('span',{'class':'t-14 t-normal'})\
                                .find('span',{'class':'visually-hidden'})
                    company = re.sub('<.*?>',"",str(company_div))
                except:
                    pass
                job_dict["Company"] = company

                #Tenure field crawled
                duration_in_months = 0
                try:
                    tenure_div = job.find('div',{'class':'display-flex flex-column full-width'})\
                                .find('span',{'class':'t-14 t-normal t-black--light'})\
                                .find('span',{'class':'visually-hidden'})
                    tenure = re.sub('<.*?>',"",str(tenure_div))
                    job_dict["Tenure"] = tenure
                    match = re.search(r'(.*) - (.*) ·(.*)', tenure)
                    if match:
                        start_date = match.group(1)
                        end_date = match.group(2)
                        period = match.group(3)

                        year_match = re.search(r'(\d*) yr.*',period)
                        months_match = re.search(r'(\d*) mo.*',period)

                        if year_match:
                            years = int(year_match.group(1))
                        else:
                            years = 0
                        if months_match:
                            months = int(months_match.group(1))
                        else:
                            months = 0
                        duration_in_months = years * 12 + months
                except:
                    pass

                total_exp_months += duration_in_months
                job_dict["Start Date"]=start_date
                job_dict["End Date"]=end_date
                job_dict["Tenure Months"]=duration_in_months


                #location field crawled
                try:
                    location_div = job.find('div',{'class':'display-flex flex-column full-width'})\
                                .find('span',{'class':'t-14 t-normal t-black--light'})\
                                .findNext('span',{'class':'t-14 t-normal t-black--light'})\
                                .find('span',{'class':'visually-hidden'})
                    location = re.sub('<.*?>',"",str(location_div))
                    location_list = location.split(",")
                    job_dict["Location"] = location
                    job_dict["City"]=location_list[0]
                except:
                    pass
                location_list = location.split(",")
                job_dict["Location"] = location
                job_dict["City"]=location_list[0]
                if(len(location_list)>=2):
                    job_dict["State"]=location_list[1]
                else:
                    job_dict["State"]=""
                if(len(location_list)>=3):
                    job_dict["Country"]=location_list[2]
                else:
                    job_dict["Country"]=""

                #Details of job description crawled
                try:
                    description_div = job.find('ul',{'class':'pvs-list'})\
                                 .find_all('div',{'class':'pvs-list__outer-container'})
                    description_string = start_date = end_date = ""
                    for describe in description_div:
                        try:
                            describe_div = describe.find('div',{'class':'display-flex full-width'})\
                                       .find('div',{'class':'display-flex align-items-center t-14 t-normal t-black'})\
                                       .find('span',{'class':'visually-hidden'})
                            description_string +=  re.sub('<.*?>',"",str(describe_div))
                        except:
                            pass       
                except:
                    pass
                job_dict["Description"] = description_string
                job_list.append(job_dict)
                text_bagofwords += designation+" "+description_string

            else:
                #Else loop for match with multiple roles in the same company
                company_div = job.find('div',{'class':'display-flex flex-column full-width align-self-center'})\
                                .find('span',{'class':'mr1 hoverable-link-text t-bold'})\
                                .find('span',{'class':'visually-hidden'})
                company = re.sub('<.*?>',"",str(company_div))

                role_list = job.find('div',{'class':'pvs-list__outer-container'})\
                            .find('ul',{'class':'pvs-list'})\
                            .find_all('div',{'class','display-flex flex-column full-width align-self-center'})


                for role in role_list:
                    job_dict ={}
                    designation_div = role.find('span',{'class':'mr1 hoverable-link-text t-bold'})\
                                .find('span',{'class':'visually-hidden'})
                    designation = re.sub('<.*?>',"",str(designation_div))

                    tenure_div = role.find('span',{'class':'t-14 t-normal t-black--light'})\
                                    .find('span',{'class':'visually-hidden'})
                    tenure = re.sub('<.*?>',"",str(tenure_div))
                    match = re.search(r'(.*) - (.*) ·(.*)', tenure)
                    if match:
                        start_date = match.group(1)
                        end_date = match.group(2)
                        period = match.group(3)

                        year_match = re.search(r'(\d*) yr.*',period)
                        months_match = re.search(r'(\d*) mo.*',period)

                        if year_match:
                            years = int(year_match.group(1))
                        else:
                            years = 0
                        if months_match:
                            months = int(months_match.group(1))
                        else:
                            months = 0

                        duration_in_months = years * 12 + months
                    else:
                        duration_in_months = 0
                        start_date = end_date = ""

                    total_exp_months += duration_in_months

                    try:
                        location_div = role.find('span',{'class':'t-14 t-normal t-black--light'})\
                                    .findNext('span',{'class':'t-14 t-normal t-black--light'})\
                                    .find('span',{'class':'visually-hidden'})
                        location = re.sub('<.*?>',"",str(location_div))
                    except:
                        pass

                    try:
                        describe_div = role.find('ul',{'class':'pvs-list'})\
                                           .find('div',{'class':'pvs-list__outer-container'})\
                                           .find('div',{'class':'display-flex full-width'})\
                                           .find('div',{'class':'display-flex align-items-center t-14 t-normal t-black'})\
                                           .find('span',{'class':'visually-hidden'})
                        description_string =  re.sub('<.*?>',"",str(describe_div))
                    except:
                        description_string = ""
                    job_dict["Company"]=company
                    job_dict["Designation"]=designation
                    job_dict["Start Date"]=start_date
                    job_dict["End Date"]=end_date
                    job_dict["Tenure Months"]=duration_in_months
                    job_dict["Tenure"]=tenure
                    job_dict["Location"]=location

                    location_list = location.split(",")
                    job_dict["Location"] = location
                    job_dict["City"]=location_list[0]
                    if(len(location_list)>=2):
                        job_dict["State"]=location_list[1]
                    else:
                        job_dict["State"] = ""
                    if(len(location_list)>=3):
                        job_dict["Country"]=location_list[2]
                    else:
                        job_dict["Country"]=""
                    job_dict["Description"]=description_string
                    text_bagofwords += designation+" "+description_string
                    job_list.append(job_dict)



        return total_exp_months, companies_count, text_bagofwords, job_list  
    

    #Code to get Skills from skills page
    def get_skills_from_skills_page(soup,text_bag_of_words):
        skill_string_list = []
        skillname_string = ""

        if(soup.find('ul',{'class':'pvs-list'}) is None):
            return skill_string_list,text_bag_of_words

        try:
            skills_list = soup.find('ul',{'class':'pvs-list'})\
                       .find_all('li',{'class':'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated'})
        except:
            return skill_string_list,text_bag_of_words

        for skill in skills_list:  
            skillname_div = skill.find('div',{'class':'display-flex flex-column full-width align-self-center'})\
                            .find('div',{'class':'display-flex align-items-center'})\
                            .find('span',{'class':'mr1 hoverable-link-text t-bold'})\
                            .find('span',{'class':'visually-hidden'})
            skillname_string = re.sub('<.*?>',"",str(skillname_div))
            text_bag_of_words += " "+skillname_string
            skill_string_list.append(skillname_string)

        return skill_string_list,text_bag_of_words

#appending and merging all the fetched data into one dataframe
    for i,row in profile_df.iterrows():
        profile_dict = {}
        profile_df_row = []
        text_bagofwords = ""
        about =""
        educ_list = []
        profilecrawl_df = pd.DataFrame([],columns=['Candidate_ID','ProfileLink','Name','City','State','Country','Total_Experience_Months',\
                                              'Companies_Count','Text_BagofWords','skills_list'])   

        profile_link = row['LinkedIn_Profile_Links']

        profile_dict["Candidate_ID"] = row['Candidate ID']
        profile_dict["Profile_Link"] = profile_link

        #Load the Main Profile Page of the candidate
        time.sleep(CALL_WAIT_TIME_SEC)
        driver.get(profile_link)
        time.sleep(PAGE_LOAD_TIME_WAIT_SEC)
        html = driver.page_source
        soup = BeautifulSoup(html)


        #Load and Crawl the Main Profile Page of the candidate to get Name, Location, About, Education
        profile_dict["Name"] = get_name_from_profile(soup)

        profile_dict["Location"] = get_location_from_profile(soup)
        profile_dict["City"] = profile_dict["Location"].split(',')[0]
        #profile_dict["State"] = profile_dict["Location"].split(',')[1]
        profile_dict["Country"] = profile_dict["Location"].split(',')[-1]


        about, educ_list, text_bagofwords = get_about_education_from_profile(soup, text_bagofwords)
        profile_dict["About"] = about
        profile_dict["Education"] = educ_list


        #Load and Crawl the Experience Page of the candidate
        time.sleep(CALL_WAIT_TIME_SEC)
        experience_page=profile_link+EXP_PAGE_PATH
        driver.get(experience_page)
        time.sleep(PAGE_LOAD_TIME_WAIT_SEC)
        html = driver.page_source
        soup = BeautifulSoup(html)

        total_exp_months, companies_count, text_bagofwords, job_list = get_experience_from_experience_page(soup,text_bagofwords)
        profile_dict["Total_Experience_Months"] = total_exp_months
        profile_dict["Companies_Count"] = companies_count
        profile_dict["Experience"] = job_list

        #Load and Crawl the Skills Page of the candidate
        time.sleep(CALL_WAIT_TIME_SEC)
        skills_page=profile_link+SKILLS_PAGE_PATH
        driver.get(skills_page)
        time.sleep(PAGE_LOAD_TIME_WAIT_SEC)
        html = driver.page_source
        soup = BeautifulSoup(html)

        skills_list, text_bagofwords = get_skills_from_skills_page(soup,text_bagofwords)
        profile_dict["Skills"] = skills_list



        #Append the Profile Dict json in file
        with open(PROFILE_JSON_FILE, 'a', encoding ='utf8') as json_file:
            json.dump(profile_dict, json_file, ensure_ascii = False,separators=(",",":"))

        #Append the Profile df in the file    
        profilecrawl_df.loc[0, 'Candidate_ID'] = profile_dict['Candidate_ID']
        profilecrawl_df.loc[0, 'ProfileLink'] = profile_dict['Profile_Link']
        profilecrawl_df.loc[0, 'Name'] = profile_dict['Name']
        profilecrawl_df.loc[0, 'City'] = profile_dict['City']
        #profilecrawl_df.loc[0, 'State'] = profile_dict['State']
        profilecrawl_df.loc[0, 'Country'] = profile_dict['Country']
        profilecrawl_df.loc[0, 'Total_Experience_Months'] = total_exp_months
        profilecrawl_df.loc[0, 'Companies_Count'] = profile_dict['Companies_Count']
        profilecrawl_df.loc[0, 'Text_BagofWords'] = text_bagofwords
        profilecrawl_df.loc[0, 'skills_list'] = profile_dict["Skills"]


        profilecrawl_df.to_csv(PROFILE_DF_FILE, mode='a', index=False, header=True)

In [9]:
def text_maching(profile_name):
    Script = pd.read_csv(r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\Outputs\linkedin_profile_dataframe.csv')
    #Converting text to lower case
    
    Script['skills_list']=Script['skills_list'].str.lower()
    # # removing punctuation
    Script['skills_list']=Script['skills_list'].astype(str).apply(lambda text: re.sub(r'[^\w\d\s]','',text))
    
    import nltk
    nltk.download('stopwords')
    # Import stopwords with nltk.
    from nltk.corpus import stopwords
    stop = stopwords.words('english')
    
    Script['skills_list'] = Script['skills_list'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    Script['skills_list']=Script['skills_list'].replace("Skills","")
    
    
    
    if profile_name == profile_name:
        Script['DATA_SCIENCE_REQUIRED_JD']="'Artificial Intelligence (AI)', 'Analytics', 'SAS', 'R', 'Microsoft Excel',  'Business Intelligence', 'Machine Learning', 'Strategy analytics', 'Analytical Skills', 'Campaign Management', 'Modeling',  'Building Automation', 'Python',  'Flask',  'Deep Learning', 'Django', 'API', 'Deployment', 'Artificial Intelligence', 'Machine Learning', 'Data Mining', 'Pattern Recognition', 'Algorithms', 'Analytics', 'Text Mining', 'MapReduce', 'Information Retrieval', 'Statistical Modeling', 'Big Data', 'Predictive Analytics', 'Optimization', 'Distributed Systems', 'Natural Language Processing', 'Image Processing', 'Computer Vision', 'Recommender Systems', 'Statistics', 'Time Series Analysis','NLP','AWS Glue', 'AWS Lambda', 'Amazon S3', 'Python (Programming Language)'"
    else:
        Script['DATA_SCIENCE_REQUIRED_JD']="'Software', 'Engineer','Java','C++','C'"
        
        
    #Converting text to lower case
    Script['DATA_SCIENCE_REQUIRED_JD']=Script['DATA_SCIENCE_REQUIRED_JD'].str.lower()
    # # removing punctuation
    Script['DATA_SCIENCE_REQUIRED_JD']=Script['DATA_SCIENCE_REQUIRED_JD'].astype(str).apply(lambda text: re.sub(r'[^\w\d\s]','',text))
    Script['DATA_SCIENCE_REQUIRED_JD']= Script['DATA_SCIENCE_REQUIRED_JD'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    
    # Match_Test=[Script['skills_list'],Script['skills_list_wo_stp']]
    Script["Match_key_word"] = Script.apply(lambda x: " ".join(i for i in x["DATA_SCIENCE_REQUIRED_JD"].split() if i in x["skills_list"]),axis=1) 

    Script["Matched_wrds"] = Script["Match_key_word"].apply(lambda n: len(n.split()))
    
    Script["Total_wrds"] = Script["DATA_SCIENCE_REQUIRED_JD"].apply(lambda n: len(n.split()))
    
    Script["Percent_match"] = Script["Matched_wrds"]/Script["Total_wrds"]
    
    #Filter where match perecent is greater than 20%
    Script = Script[Script['Percent_match'] >= 0.05] 
    
    Script.to_csv(r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\Outputs\Shortlisted_profiles.csv', header=True, index=False)
    
    Script_output = Script[['Candidate_ID','ProfileLink','Name','Percent_match']]
    print(tabulate(Script_output,headers = 'keys',tablefmt = 'psql'))
    #print(Script_output)

In [10]:
import pickle

In [11]:

#created a pickel to call the model
def prediction(X):
    with open(r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\model1.pkl', 'rb') as f:
        clf2 = pickle.load(f)

    clf2.predict(X[0:1])
    if (clf2.predict(X[0:1])[0]) == 1:
        print("Candidate will accept the offer")
    else:
        print("Candidate will most probably reject the offer")

In [12]:
#creating the buttons to call the functions

button_send_2 = widgets.Button(
                description='Submit',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output_2 = widgets.Output()

def on_button_clicked(event):
    with output_2:
        clear_output()
        print("Fetching LinkedIn Profiles Urls")
        country_name=location.value
        #print(country_name)
        profile_name=job_profile.value
        #profiles_required = profiles_required.value
        #{grand.value}
        get_urls(country_name,profile_name)
        #print("url found")
        get_scrapped_data()
        #print("profiles scraped")
        text_maching(profile_name)
        #print("text matching done")

button_send_2.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send_2, output_2])

In [13]:
from tabulate import tabulate

In [14]:
button_send = widgets.Button(
                description='Predict',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

button_send_1 = widgets.Button(
                description='Submit Candidate ID',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()
output_1 = widgets.Output()


def candidate_id(event):
    with output_1:
        clear_output()
        shortlisted = pd.read_csv(r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\Outputs\Shortlisted_profiles.csv')
        shortlisted['Candidate_ID'] = shortlisted['Candidate_ID'].astype(str)
        shortlisted = shortlisted[shortlisted['Candidate_ID']==Candidate_ID.value]
        print(tabulate(shortlisted[['ProfileLink','Total_Experience_Months','Companies_Count','Percent_match']],headers = 'keys',tablefmt = 'psql'))
        #print(shortlisted[['ProfileLink','Total_Experience_Months','Companies_Count','Percent_match']])
        

button_send_1.on_click(candidate_id)
vbox_result_1 = widgets.VBox([button_send_1, output_1])


def on_button_clicked(event):
    with output:
        clear_output()
        #print("Submit successfully")
        shortlisted = pd.read_csv(r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\Outputs\Shortlisted_profiles.csv')
        shortlisted['Candidate_ID'] = shortlisted['Candidate_ID'].astype(str)
        shortlisted = shortlisted[shortlisted['Candidate_ID']==Candidate_ID.value]
        X = pd.read_csv(r'C:\Users\Deepak Gupta\OneDrive - Indian School of Business\Term 2\Foundational Project 1\Group Project\sample.csv')        
        
        X['Duration_to_accept_offer'] = X['Duration_to_accept_offer'].astype(str)
        X['Duration_to_accept_offer'] = [Duration_to_accept_offer.value]
        X['Notice_period'] = [Notice_period.value]
        X['Percent_difference_CTC'] = [Percent_difference_CTC.value]
        X['Rex_in_Yrs'] = [shortlisted['Total_Experience_Months']/12]
        X['Companies_Count'] = [shortlisted['Companies_Count']]
        X['Proximity_to_Job_Location'] = [Proximity_to_Job_Location.value]
        X['Age'] = [Age.value]
        X['Match_Score'] = [shortlisted['Percent_match']]
        X['DOJ_Extended_Yes'] = [DOJ_Extended_Yes.value]
        X['Offered_band_E1'] = 0
        X['Offered_band_E2'] = 0
        X['Offered_band_E3'] = 0
        if Offered_band.value == "Offered_band_E1":
             X['Offered_band_E1'] = 1
        elif Offered_band.value == "Offered_band_E2":
             X['Offered_band_E2'] = 1
        elif Offered_band.value == "Offered_band_E3":
             X['Offered_band_E3'] = 1
        else:
            X['Offered_band_E3'] = 1
        
        X['Joining_Bonus_Yes'] = [Joining_Bonus_Yes.value]
        X['Candidate_relocate_actual_Yes'] = [Candidate_relocate_actual_Yes.value]
        X['Gender_Male'] = [Gender_Male.value]
        X['LOB_BFSI'] = 0
        X['LOB_CSMP'] = 0
        X['LOB_EAS'] = 0
        X['LOB_ERS'] = 0
        X['LOB_ETS'] = 0
        X['LOB_Healthcare'] = 0
        X['LOB_INFRA'] = 0
        X['LOB_MMS'] = 0
        if LOB.value == "LOB_BFSI":
            X['LOB_BFSI'] = 1
        elif LOB.value == "LOB_CSMP":
            X['LOB_CSMP'] = 1
        elif LOB.value == "LOB_EAS":
            X['LOB_EAS'] = 1
        elif LOB.value == "LOB_ERS":
            X['LOB_ERS'] = 1
        elif LOB.value == "LOB_ETS":
            X['LOB_ETS'] = 1
        elif LOB.value == "LOB_Healthcare":
            X['LOB_Healthcare'] = 1
        elif LOB.value == "LOB_INFRA":
            X['LOB_INFRA'] = 1
        elif LOB.value == "LOB_MMS":
            X['LOB_MMS'] = 1
        else:
            X['LOB_MMS'] = 0
        
        prediction(X)
        print("prediction is done")


button_send.on_click(on_button_clicked)

vbox_result_2 = widgets.VBox([button_send, output])

In [15]:
#giving the headers of each page

text_0 = widgets.HTML(value="<h1>Get Profiles</h1>")
text_1 = widgets.HTML(value="<h1>Enter Candidate ID</h1>")
text_2 = widgets.HTML(value="<h1>Use for prediction</h1>")

#assigning respective widgets to respective pages
vbox_text = widgets.VBox([text_0, job_profile, location,profiles_required,vbox_result])
vbox_text_1 = widgets.VBox([text_1, Candidate_ID ,vbox_result_1])
vbox_text_2 = widgets.VBox([text_2, Candidate_ID ,Percent_difference_CTC,Proximity_to_Job_Location,Age,Duration_to_accept_offer,Notice_period,DOJ_Extended_Yes,Offered_band,Joining_Bonus_Yes,Candidate_relocate_actual_Yes,Gender_Male,LOB,vbox_result_2])

In [16]:
#displaying the final deployment

page = widgets.HBox([vbox_text])
page_1 = widgets.HBox([vbox_text_1])
page_2 = widgets.HBox([vbox_text_2])
display(page,page_1,page_2)